# CFT2018 contest baseline

In [1]:
import pandas as pd
import numpy as np

Загрузим данные.

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
train.shape, test.shape

((1991104, 5), (2767639, 3))

In [4]:
train.head()

,id,fullname,country,target,fullname_true
0,0,AKHMEDOV YGURIY,РОССИЯ,1,AKHMEDOV YURIY
1,1,ФОЗИЛОВ РАМИЛЬ ГУЛЛОВИЧ,РОССИЯ,1,ФОЗИЛОВ РАМИЛЬ ГУЛОВИЧ
2,2,ГОИБОВ АХЛИДДИН ШАМСУДИНОВИЧ,РОССИЯ,0,NaN
3,3,ХУСНЕУТДИНОВА МАРГАРИТА ФАХИМОВНА,РОССИЯ,1,ХУСНУТДИНОВА МАРГАРИТА ФАХИМОВНА
4,4,НОВОКШОНОВА ИННА ВЛАДИМИРОВНА,РОССИЯ,0,NaN


In [5]:
test.head()

,id,fullname,country
0,0,ХУДАШКУРОВА ГУЛЗХОДА БЕРДИЕВНА,УЗБЕКИСТАН
1,1,СВЕЖЕТЬФЛОГИСТОН АРСЕН,РОССИЯ
2,2,ГУЛОМОВА СОЖИДА САНАЕВНА,УЗБЕКИСТАН
3,3,КАМПЫШЕВА ГУЛЯИМ БЕЙСЕМБАЕВНА,КАЗАХСТАН
4,4,OROSUMEBTOV MIRLAN,РОССИЯ


### Определение корректности ФИО

Начнём с первой задачи. Сгенерируем признаки на основе tf-idf-преобразования. Используем две версии: по словам (чтобы поймать популярные имена) и по тройкам символов (чтобы поймать опечатки).

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
vectorizer_word = TfidfVectorizer(analyzer='word', ngram_range=(1, 1))

train_word = vectorizer_word.fit_transform(train.fullname)
test_word = vectorizer_word.transform(test.fullname)

In [8]:
vectorizer_char = TfidfVectorizer(analyzer='char', ngram_range=(3, 3), min_df=5)

train_char = vectorizer_char.fit_transform(train.fullname)
test_char = vectorizer_char.transform(test.fullname)

Объединим признаки в одну матрицу.

In [9]:
from scipy.sparse import hstack
from scipy.sparse import csr_matrix

s_train = hstack([train_word, train_char])
s_test = hstack([test_word, test_char])

Оценим качество на отложенной выборке. Обучать на таких признаках будем логистическую регрессию.

In [10]:
from sklearn.model_selection import train_test_split

y_train = train['target'].values
X_tr, X_ts, y_tr, y_ts = train_test_split(s_train, y_train, test_size=0.2, random_state=322)

In [11]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_tr, y_tr)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [12]:
from sklearn.metrics import recall_score, precision_score, f1_score

y_predict = model.predict(X_ts)

print('Recall:', recall_score(y_ts, y_predict, average='macro'))
print('Precision:', precision_score(y_ts, y_predict, average='macro'))
print('F1:', f1_score(y_ts, y_predict, average='macro'))

Recall: 0.8946307143193054
Precision: 0.9277103543759097
F1: 0.9096538430602945


Обучим финальную версию для тестовой выборки и сделаем предсказание.

In [13]:
model = LogisticRegression()
model.fit(s_train, y_train)

test['target'] = model.predict(s_test)

### Исправление опечаток

Для коррекции опечаток воспользуемся open-source библиотекой ([github](https://github.com/mammothb/symspellpy)). Можно установить через pip. 

In [14]:
import symspellpy
symspell = symspellpy.SymSpell()

Подготовим обучающую выборку для корректора. Добавим туда весь корректный train. На выходе нам нужно отдать файл с частотами слов.

In [15]:
train.loc[train.target != 1, 'fullname_true'] = train.loc[train.target != 1, 'fullname'] 
dicts = [name for person in train.fullname_true for name in person.split(' ')]

from collections import Counter
name_freq = Counter(dicts)

In [16]:
with open('dictionary.txt', 'w') as f:
    for name, freq in name_freq.items():
        f.write('{} {}\n'.format(name, freq))

In [17]:
!head dictionary.txt

AKHMEDOV 381
YURIY 98
ФОЗИЛОВ 783
РАМИЛЬ 2383
ГУЛОВИЧ 190
ГОИБОВ 482
АХЛИДДИН 753
ШАМСУДИНОВИЧ 183
ХУСНУТДИНОВА 42
МАРГАРИТА 1923


Загрузим словарь в модель.

In [18]:
symspell.load_dictionary('dictionary.txt', term_index=0, count_index=1)

True

Будем проводить коррекцию по словам.

In [19]:
def correct(s):
    def correct_word(w):
        tmp = symspell.lookup(w, symspellpy.Verbosity.CLOSEST)
        if len(tmp):
            return tmp[0].term.upper()
        else:
            return w

    return ' '.join([correct_word(word) for word in s.split(' ')])

In [20]:
%%time
correct('КАРАБОЗОВа ЛАТИФ АЛИМАМАДОВИЧ')

CPU times: user 1.85 ms, sys: 74 µs, total: 1.93 ms
Wall time: 1.97 ms


'КАРАБОЗОВ ЛАТИФ АЛИМАМАДОВИЧ'

Посчитаем качество на train-выборке (переобученное!)

In [21]:
train_1 = train.loc[train.target == 1].copy()
train_1['fullname_corrected'] = train_1.fullname.apply(correct)

In [22]:
np.mean(train_1.fullname_true == train_1.fullname_corrected)

0.8272597104120801

Скорректируем тестовую выборку.

In [23]:
test['fullname_true'] = None

test.loc[test.target == 1, 'fullname_true'] = test.loc[test.target == 1, 'fullname'].apply(correct)

Сохраним итоговый файл.

In [24]:
test[['id', 'target', 'fullname_true']].to_csv('submission_baseline.csv', index=False)